In [16]:
import string
import requests
import requests_cache
from requests_cache import CachedSession
from selenium import webdriver 
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import time
import json
import re
import pandas as pd 
from datetime import datetime, timedelta 
from requests_html import HTMLSession

In [17]:
#Tao cache
def check_cached(response: requests.Response) ->bool :
    if(response.text.find('"incomplete_results":true')==-1):
        return True
    else:
        return False
    
requests_cache.install_cache(expire_after = -1, filter_fn = check_cached)

In [34]:
index = []
country = []
total_cases = []
total_deaths = []
total_recovered = []
active_cases = []
serious_critical = []
tot_cases = []
deaths = []
total_tests = []
tests = []
population = []
region = []
case_every = []
death_every = []
test_every = []

In [35]:
url = "https://www.worldometers.info/coronavirus/#nav-yesterday"

In [36]:
driver = webdriver.Chrome()

In [37]:
driver.get(url)
time.sleep(5)

In [ ]:
table = driver.find_element_by_xpath("/html/body/div[3]/div[3]/div/div[6]/div[3]/div/table")

In [24]:
soup = BeautifulSoup(driver.page_source, "html.parser")

In [25]:
odd_class = soup.find_all('tr', class_='odd')

In [26]:
for i in odd_class:
    att = i.find_all('td')
    if len(att)!=16:
        continue
    else:
        if att[0].text == '' or att[1].text == '':
            continue
        else:
            for j in range(16):
                if (j==0):
                    index.append(att[j].text)
                elif(j==1):
                    country.append(att[j].text)
                elif(j==2):
                    total_cases.append(att[j].text)
                elif(j==3):
                    total_deaths.append(att[j].text)
                elif(j==4):
                    total_recovered.append(att[j].text)
                elif(j==5):
                    active_cases.append(att[j].text)
                elif(j==6):
                    serious_critical.append(att[j].text)
                elif(j==7):
                    tot_cases.append(att[j].text)
                elif(j==8):
                    deaths.append(att[j].text)
                elif(j==9):
                    total_tests.append(att[j].text)
                elif(j==10):
                    tests.append(att[j].text)
                elif(j==11):
                    population.append(att[j].text)
                elif(j==12):
                    region.append(att[j].text)
                elif(j==13):
                    case_every.append(att[j].text)
                elif(j==14):
                    death_every.append(att[j].text)
                elif(j==15):
                    test_every.append(att[j].text)

In [27]:
data = {'Country': country, 'Total Cases': total_cases, 'Total Death': total_deaths,
       'Total Recovered': total_recovered, 'Active Cases': active_cases, 'Serious Critical': serious_critical,
       'Tot cases/1M pop': tot_cases, 'Deaths/1M pop': deaths, 'Total tests': total_tests, 'Tests/ 1M pop': tests,
       'region': region, 'population': population, '1 Case every X ppl': case_every, '1 death every X ppl': death_every,
       '1 Test every X ppl': test_every} 

In [28]:
df = pd.DataFrame(data)

In [29]:
df

,Country,Total Cases,Total Death,Total Recovered,Active Cases,Serious Critical,Tot cases/1M pop,Deaths/1M pop,Total tests,Tests/ 1M pop,region,population,1 Case every X ppl,1 death every X ppl,1 Test every X ppl


In [14]:
df.to_csv('newn_data.csv')

In [15]:
df.drop_duplicates()

,Country,Total Cases,Total Death,Total Recovered,Active Cases,Serious Critical,Tot cases/1M pop,Deaths/1M pop,Total tests,Tests/ 1M pop,region,population,1 Case every X ppl,1 death every X ppl,1 Test every X ppl
0,USA,"82,416,687",,"1,016,159",,"80,276,197",,"1,124,331","1,436","246,399","997,115,318","3,038","2,981,044","334,485,290",North America
1,India,"43,047,594",,"522,006",,"42,513,248",,"12,340",698,"30,653","832,506,755",372,"592,807","1,404,347,026",Asia
2,France,"27,816,299",,"144,387",,"25,301,145",,"2,370,767","1,541","424,465","260,504,402","2,203","3,975,184","65,532,665",Europe
3,UK,"21,863,944",,"171,878",,"20,498,917",,"1,193,149",355,"319,061","512,140,269","2,508","7,473,679","68,525,857",Europe
4,S. Korea,"16,583,220","+111,280","21,520",+166,N/A,N/A,N/A,808,"322,955","15,804,065",419,"307,781","51,348,351",Asia
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
329,Chad,"7,396",+18,193,+1,"4,874",,"2,329",,428,"191,341",11,"11,079","17,270,935",Africa
330,Vanuatu,"6,386",+54,7,,"5,445",+145,934,5,"19,955","24,976",22,"78,046","320,016",Australia/Oceania
331,Sao Tome and Principe,"5,953",+5,73,,"5,875",+3,5,,"26,287","29,036",322,"128,217","226,459",Africa
334,Cook Islands,"3,941",,,,"3,389",,552,,"224,010","14,341",,"815,154","17,593",Australia/Oceania
